# car mask

In [1]:
specified_sequence_id = "0022"

In [11]:
import os, sys

sys.path.append(os.getcwd())
import argparse
import os
import copy
import imageio
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision.ops import box_convert
from huggingface_hub import hf_hub_download
from tqdm import tqdm
from termcolor import colored
from glob import glob

# Grounding DINO
import groundingdino.datasets.transforms as T
from groundingdino.models import build_model
from groundingdino.util import box_ops
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from groundingdino.util.inference import annotate, load_image, predict
import supervision as sv

# segment anything
from segment_anything import build_sam, SamPredictor 
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [12]:

def setup(args):
    # ======================== Load Grounding DINO model ========================
    print(colored('Load Grounding DINO model', 'green'))
    def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
        cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

        args = SLConfig.fromfile(cache_config_file) 
        model = build_model(args)
        args.device = device

        cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
        checkpoint = torch.load(cache_file, map_location='cpu')
        log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
        print("Model loaded from {} \n => {}".format(cache_file, log))
        _ = model.eval()
        return model   

    # Use this command for evaluate the Grounding DINO model
    # Or you can download the model by yourself
    ckpt_repo_id = "ShilongLiu/GroundingDINO"
    ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
    ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"
    global groundingdino_model 
    groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename)

    # ======================== Load Segment Anything model ========================
    print(colored('Load SAM model', 'green'))
    sam = build_sam(checkpoint=args.sam_checkpoint)
    sam.cuda()
    global sam_predictor
    sam_predictor = SamPredictor(sam)

image_filename_to_cam = lambda x: int(x.split('.')[0][-1])
image_filename_to_frame = lambda x: int(x.split('.')[0][:6])


In [13]:

def add_to_mask_dict(masks_dict, mask_path):
    basename = os.path.basename(mask_path)
    cam = image_filename_to_cam(basename)
    frame = image_filename_to_frame(basename)
    mask = cv2.imread(mask_path) 
    if frame not in masks_dict:
        masks_dict[frame] = [None] * 3 # FRONT_LEFT, FRONT, FRONT_RIGHT 1, 0, 2
    if cam == 1:
        masks_dict[frame][0] = mask
    elif cam == 0:
        masks_dict[frame][1] = mask
    elif cam == 2:
        masks_dict[frame][2] = mask
    


In [22]:
def segment_with_text_prompt(datadir, BOX_TRESHOLD, TEXT_TRESHOLD, ignore_exists):
    save_dir = os.path.join(datadir, 'car_mask_1')
    os.makedirs(save_dir, exist_ok=True)

    image_dir = os.path.join(datadir, 'images')
    image_files = glob(image_dir + "/*.jpg") 
    image_files += glob(image_dir + "/*.png")
    image_files = sorted(image_files)
    
    masks_dict = dict()
    for image_path in tqdm(image_files):
        image_base_name = os.path.basename(image_path)
        output_mask = os.path.join(save_dir, image_base_name)
                        
        if os.path.exists(output_mask) and ignore_exists:
            add_to_mask_dict(masks_dict, output_mask)
            print(f'{output_mask} exists, skip')
            continue
        
        cam = image_filename_to_cam(image_base_name)
        box_threshold = BOX_TRESHOLD[cam]
        
        image_source, image = load_image(image_path)
        boxes, logits, phrases = predict(
            model=groundingdino_model, 
            image=image, 
            caption='car. tricycle. motorcycle. pedestrian.', 
            box_threshold=box_threshold, 
            text_threshold=TEXT_TRESHOLD
        )

        print(f'detecting {boxes.shape[0]} boxed of car in {image_path}, box_threshold: {box_threshold}, logits: {logits}')
        if boxes.shape[0] != 0:
            H, W, _ = image_source.shape
            boxes = box_ops.box_cxcywh_to_xyxy(boxes)
            boxes_xyxy = boxes * torch.Tensor([W, H, W, H])
            # assume that the box prompt for sky should be close to the top edge of the image
            #  --------------  top edge 
            # | x ----       |
            # | |    |       |
            # |  ----x       |
            #  --------------
            # boxes_mask = boxes_xyxy[:, 1] < 100 # 100 pixels
            # boxes_xyxy = boxes_xyxy[boxes_mask]
        else:
            boxes_xyxy = []
        
        num_boxes = len(boxes_xyxy)

        if num_boxes == 0:                
            mask = np.zeros_like(image_source[..., 0])
        else:
            sam_predictor.set_image(image_source)
            transformed_boxes = sam_predictor.transform.apply_boxes_torch(boxes_xyxy, image_source.shape[:2]).cuda()
            masks, _, _ = sam_predictor.predict_torch(
                        point_coords = None,
                        point_labels = None,
                        boxes = transformed_boxes,
                        multimask_output = False,
                    )
            
            torch.cuda.empty_cache()
            mask_final = torch.zeros_like(masks[0, 0]).bool()
            for mask in masks[:, 0]:
                mask_final = mask_final | mask.bool()
                
            mask = mask_final.cpu().numpy()
            
        cv2.imwrite(output_mask, mask * 255)
        add_to_mask_dict(masks_dict, output_mask)


In [15]:
class Args:
    def __init__(self):
        # self.datadir = f'data/dair-v2x/exp/{specified_sequence_id}_0_original'
        self.datadir = f'/mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud'
        self.box_threshold = [0.3]
        self.text_threshold = 0.25
        self.ignore_exists = ''
        self.sam_checkpoint = '/mnt/xuhr/GroundingDINO/weights/sam_vit_h_4b8939.pth'
        
        
args = Args()

In [6]:
setup(args)


Load Grounding DINO model


final text_encoder_type: bert-base-uncased


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Model loaded from /home/jumpserver/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
Load SAM model


In [23]:

assert isinstance(args.box_threshold, list)
if len(args.box_threshold) == 1:
    box_threshold = [args.box_threshold[0]] * 5
else:
    assert len(args.box_threshold) == 5
    box_threshold = args.box_threshold
print('box_threshold: ', box_threshold)
    


box_threshold:  [0.3, 0.3, 0.3, 0.3, 0.3]


In [24]:
segment_with_text_prompt(
    datadir=args.datadir, 
    BOX_TRESHOLD=box_threshold,
    TEXT_TRESHOLD=args.text_threshold,
    ignore_exists=args.ignore_exists,
)

  0%|          | 0/250 [00:00<?, ?it/s]FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.


detecting 4 boxed of car in /mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud/images/000000_0.jpg, box_threshold: 0.3, logits: tensor([0.4723, 0.4219, 0.3133, 0.4144])


  0%|          | 1/250 [00:00<02:49,  1.47it/s]

detecting 12 boxed of car in /mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud/images/000000_1.jpg, box_threshold: 0.3, logits: tensor([0.7056, 0.6614, 0.5590, 0.5662, 0.5308, 0.4904, 0.4445, 0.3880, 0.3363,
        0.3124, 0.3082, 0.3367])


  1%|          | 2/250 [00:01<02:47,  1.48it/s]

...

 99%|█████████▉| 248/250 [03:50<00:02,  1.05s/it]

detecting 2 boxed of car in /mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud/images/000124_0.jpg, box_threshold: 0.3, logits: tensor([0.6785, 0.3807])


100%|█████████▉| 249/250 [03:51<00:00,  1.00it/s]

detecting 20 boxed of car in /mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud/images/000124_1.jpg, box_threshold: 0.3, logits: tensor([0.7076, 0.7382, 0.5750, 0.5268, 0.6286, 0.4996, 0.4444, 0.4212, 0.4292,
        0.5068, 0.4088, 0.3798, 0.3907, 0.3332, 0.3160, 0.3474, 0.3598, 0.3295,
        0.3195, 0.4043])


100%|██████████| 250/250 [03:52<00:00,  1.08it/s]


In [25]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# 定义路径
image_folder = args.datadir + "/images"  # 原始图片文件夹
mask_folder = args.datadir + "/car_mask_1"  # mask文件夹
output_folder = args.datadir + "/car_mask_visualization_1"  # 输出文件夹

# 创建输出文件夹
os.makedirs(output_folder, exist_ok=True)

# 获取所有mask文件
mask_files = sorted(os.listdir(mask_folder))

# 处理每个文件
for mask_file in tqdm(mask_files):
    if not mask_file.endswith('.jpg'):  # 假设mask是png格式
        continue
        
    # 读取对应的图片和mask
    image_path = os.path.join(image_folder, mask_file.replace('.png', '.jpg'))  # 假设原图是jpg格式
    mask_path = os.path.join(mask_folder, mask_file)
    
    if not os.path.exists(image_path):
        print(f"找不到对应的图片: {image_path}")
        continue
        
    # 读取图片和mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # 创建彩色mask（使用红色）
    colored_mask = np.zeros_like(image)
    colored_mask[mask > 0] = [0, 0, 255]  # BGR格式，红色
    
    # 叠加mask到原图（透明度0.5）
    alpha = 0.5
    overlay = cv2.addWeighted(image, 1, colored_mask, alpha, 0)
    
    # 保存结果
    output_path = os.path.join(output_folder, mask_file)
    cv2.imwrite(output_path, overlay)

print("处理完成！结果保存在:", output_folder)

100%|██████████| 250/250 [00:07<00:00, 34.25it/s]

处理完成！结果保存在: /mnt/zhangsn/data/V2X-Seq-SPD-Processed/0022_exp/0022_0_original_all_seperate_with_cooperative_pointcloud/car_mask_visualization_1
